# Human and synthetic text dataset wrangling

## Introduction

We have 5 target datasets. The plan it to get them downloaded and saved locally. Read into Python as appropriate and combined into a unified dataset. Here are the target datasets:

1. [Hans 2024](https://github.com/ahans30/Binoculars/tree/main), referred to as `hans`. Source: GitHub.
2. [AI vs human text](https://www.kaggle.com/datasets/shanegerami/ai-vs-human-text), referred to as `gerami`. Source: Kaggle.
3. [Human vs. LLM text corpus](https://www.kaggle.com/datasets/starblasters8/human-vs-llm-text-corpus), referred to as `grinberg`. Source: Kaggle.
4. [Human-ChatGPT texts](https://github.com/HarshOza36/Detection-Of-Machine-Generated-Text/tree/master), referred to as `gaggar`. Source: GitHub.
5. [ai-text-detection-pile](https://huggingface.co/datasets/artem9k/ai-text-detection-pile), referred to as `yatsenko`. Source: HuggingFace.

## 1. Raw data acquisition

First, download the raw data from each source so that we have a local copy archived.

### 1.1. Hans

This dataset comes in 6 JSON-lines formatted files. One for each combination of data source and generating model.

In [ ]:
generating_models=['falcon7','llama2_13']
data_sources=['cnn','cc_news','pubmed']
base_url='https://github.com/ahans30/Binoculars/tree/main/datasets/core'